In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import json
import os
import shutil
import random
from PIL import Image

print('Using PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.')
    device = torch.device('cpu')

Using PyTorch version: 2.3.1
No GPU found, using CPU instead.


In [2]:

# Définir une nouvelle transformation pour convertir les images en niveaux de gris
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Redimensionner les images à 112x112
    transforms.Grayscale(),  # Convertir les images en niveaux de gris
    transforms.ToTensor(),  # Convertir les images en tenseurs PyTorch
])


# Télécharger et charger le jeu de données Flowers102
dataset = datasets.Flowers102(root='./data', 
                              split='train', 
                              transform=transform, 
                              download=True)

# Séparer les données en ensemble d'entraînement et de test
# Calculer la taille de chaque sous-dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Diviser le dataset en ensemble d'entraînement et de test
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Créer des DataLoader pour les ensembles d'entraînement et de test
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Afficher les tailles des datasets pour vérification
print(f'Taille du dataset d\'entraînement: {len(train_dataset)}')
print(f'Taille du dataset de test: {len(test_dataset)}')

Taille du dataset d'entraînement: 816
Taille du dataset de test: 204


In [3]:
class TwoLayerMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(112*112, 100),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(100, 102)
        )

    def forward(self, x):
        return self.layers(x)

model = TwoLayerMLP()
print(model)

assert len(model.layers) > 0, "ERROR: You need to write the missing model definition above!"
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

def correct(output, target):
    predicted_digits = output.argmax(1)                            # choisir le chiffre avec la plus grande sortie du réseau
    correct_ones = (predicted_digits == target).type(torch.float)  # 1.0 pour correct, 0.0 pour incorrect
    return correct_ones.sum().item()                               # compter le nombre de bons

TwoLayerMLP(
  (layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=12544, out_features=100, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=100, out_features=102, bias=True)
  )
)


In [4]:
def train(data_loader, model, criterion, optimizer):
    model.train()

    num_batches = len(data_loader)
    num_items = len(data_loader.dataset)

    total_loss = 0
    total_correct = 0
    for data, target in data_loader:
        # Copier les données et les cibles sur le GPU
        data = data.to(device)
        target = target.to(device)

        # Effectuer une passe avant
        output = model(data)

        # Calculer la perte
        loss = criterion(output, target)
        total_loss += loss

        # Compter le nombre de chiffres corrects
        total_correct += correct(output, target)

        # Rétropropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    train_loss = total_loss/num_batches
    accuracy = total_correct/num_items
    print(f"Average loss: {train_loss:7f}, accuracy: {accuracy:.2%}")

epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_loader, model, criterion, optimizer)

Training epoch: 1
Average loss: 4.710788, accuracy: 1.10%
Training epoch: 2
Average loss: 4.621845, accuracy: 1.10%
Training epoch: 3
Average loss: 4.620938, accuracy: 0.98%
Training epoch: 4
Average loss: 4.615074, accuracy: 0.86%
Training epoch: 5
Average loss: 4.603566, accuracy: 1.59%
Training epoch: 6
Average loss: 4.590473, accuracy: 1.72%
Training epoch: 7
Average loss: 4.568855, accuracy: 2.33%
Training epoch: 8
Average loss: 4.561790, accuracy: 1.96%
Training epoch: 9
Average loss: 4.535686, accuracy: 2.08%
Training epoch: 10
Average loss: 4.525084, accuracy: 1.84%


In [5]:
def test(test_loader, model, criterion):
    model.eval()

    num_batches = len(test_loader)
    num_items = len(test_loader.dataset)

    test_loss = 0
    total_correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            # Copier les données et les cibles sur le GPU
            data = data.to(device)
            target = target.to(device)

            # Effectuer une passe avant
            output = model(data)

            # Calculer la perte
            loss = criterion(output, target)
            test_loss += loss.item()

            # Compter le nombre de chiffres corrects
            total_correct += correct(output, target)

    test_loss = test_loss/num_batches
    accuracy = total_correct/num_items

    print(f"Testset accuracy: {100*accuracy:>0.1f}%, average loss: {test_loss:>7f}")

test(test_loader, model, criterion)

Testset accuracy: 0.0%, average loss: 4.650013
